영어 -> feature 변환 -> 머신러닝모델 -> 

한글 -> 한글 형태소분석기 -> feature변환 -> 머신러닝모델 -> 

In [104]:
# 경고 무시
import warnings
warnings.filterwarnings(action='ignore')

# 네이버 영화평 감성 분석

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [106]:
# 데이터 데이터프레임으로 불러오기

train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",sep = '\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",sep = '\t')

train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


## 1. 데이터 전처리

### - train data set

In [107]:
train_df.document.unique()

array(['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
       '너무재밓었다그래서보는것을추천한다', ..., '이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?',
       '청춘 영화의 최고봉.방황과 우울했던 날들의 자화상', '한국 영화 최초로 수간하는 내용이 담긴 영화'],
      dtype=object)

In [108]:
# 중복여부 확인
train_df.document.nunique() # 데이터에 NULL값이 있으면 UNIQUE로 파악되지 않음
# 따라서 NULL값도 확인해야함

146182

In [109]:
# 중복데이터 제거
train_df.drop_duplicates(subset =['document'],inplace = True)

In [110]:
# null값 확인
train_df.isnull().sum() # null값 있네

id          0
document    1
label       0
dtype: int64

In [111]:
# null 데이터 제거
train_df = train_df.dropna(how ='any')
train_df.shape

(146182, 3)

In [112]:
# 데이터 분포 확인
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [113]:
# 중복데이터 제거
train_df.drop_duplicates(subset =['document'],inplace = True)

### - test data set

In [114]:
test_df.document.unique()

array(['굳 ㅋ', 'GDNTOPCLASSINTHECLUB',
       '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', ...,
       '그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다',
       '절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네', '마무리는 또 왜이래'],
      dtype=object)

In [115]:
test_df.document.nunique()

49157

In [116]:
# null값 확인
test_df.isnull().sum() 

id          0
document    3
label       0
dtype: int64

In [117]:
# null 데이터 제거
train_df = train_df.dropna(how ='any')
train_df.shape

(146182, 3)

In [118]:
# 데이터 분포 확인
test_df.label.value_counts()

1    25173
0    24827
Name: label, dtype: int64

## 2. 텍스트 전처리

### - train data set

In [119]:
# 한글[ㄱ-ㅎㅏ-ㅣ가-힣]과 공백[ ]이외에는 제거 : [^ㄱ-ㅎㅏ-ㅣ가-힣 ]
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [120]:
# ''데이터는 Nan으로 변환한 후 제거
train_df['document'].replace('',np.nan, inplace = True)
train_df.document.isnull().sum()

391

In [121]:
train_df = train_df.dropna(how="any")
train_df.shape

(145791, 3)

### - test dataset

In [122]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df['document'].replace('',np.nan, inplace = True)
test_df = test_df.dropna(how="any")
test_df.shape

(49726, 3)

In [123]:
train_df.to_csv('naver_movie_train.tsv',sep= '\t')
test_df.to_csv('naver_movie_test.tsv',sep= '\t')

## 3. 한글처리

In [124]:
from konlpy.tag import Okt
okt = Okt()

In [125]:
stopwords =['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자',
            '에','와','한','하다','울','ㅋㅋ','ㅜㅜ','ㅎㅎ']

In [126]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text) #형태소 분석기 

# 형태소 분석기를 통하면 리스트로 값이 나옴
# 이후 CountVectorizer를 이용하려면 문장을 넣어주어야하므로,
# 리스트를 .join()를 이용해 문장으로 만들어 주어야한다.

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [127]:
okt.morphs(text, stem =True)# 부사->형용사으로 바꾸어줌

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [128]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

In [129]:
X_test =[]
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem = True)
    temp_X = ''.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

In [130]:
y_train = train_df.label.values
y_test = test_df.label.values

## 4. Feature 변환, 모델/ 학습/ 예측 / 평가

In [131]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [132]:
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [133]:
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

LogisticRegression()

In [144]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test,pred)

0.5069983509632787

## 5. 실제테스트

In [135]:
review1 = "아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~" # 긍정
review2 = "메세지와 작위성의 불협화음!!!" # 부정

### -  review1

In [136]:
#전처리 case1
review1 = review1.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
review1 # 작동안함

'아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'

In [137]:
#전처리 case2 
import re
review1 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review1)
review1 #정상작동 

'아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요'

In [138]:
morphs = okt.morphs(review1)
review = ' '.join([word for word in morphs if not word in stopwords])
review

'아름다운 음악 아름다운 풍 광 그렇지 못 현실 찡하네요'

In [139]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]# 1 : 긍정의미

1

### -  review2

In [140]:
review1 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review2)
morphs = okt.morphs(review2)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]# 0: 부정의미

0

In [147]:
reviews = ["아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~" ,
            "메세지와 작위성의 불협화음!!!"]
reviews[0]

'아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'

In [150]:
import re
review3=[]
for i in range(2):review3.append(re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", reviews[i]))
review3 #정상작동 

['아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요', '메세지와 작위성의 불협화음']

In [157]:
morphs, review, review_cv,pred = [],[],[],[]
for i in range(2):
    morphs.append( okt.morphs(review3[i])) 
    review.append(' '.join([word for word in morphs[i] if not word in stopwords]) )
    review_cv.append(cvect.transform([review]) ) 
    pred.append(lr.predict(review_cv[i])) 
pred[0]

AttributeError: 'list' object has no attribute 'lower'

In [154]:
morphs

['메세지', '와', '작위', '성의', '불협화음']

In [ ]:
review_list= []
for i in range(2):
    review3.append(re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", reviews[i]))

    morphs.append( okt.morphs(review3[i])) 
    review.append(' '.join([word for word in morphs if not word in stopwords]) )

In [ ]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]# 0: 부정의미

## 6. GridSearchCV로 최적 파라미터 찾기

In [158]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [167]:
pipeline = Pipeline([
    ('cvect',CountVectorizer()),
    ('lr',LogisticRegression())
])
params = {
    'cvect__ngram_range':[(1,1),(1,2)],
    'cvect__max_df':[0.9, 0.99],
    'cvect__min_df':[1,3],
    'lr__C':[1,5]
}

In [168]:
grid_pipe = GridSearchCV(
    pipeline, param_grid = params, cv = 3, scoring='accuracy', n_jobs =-1
)
%time grid_pipe.fit(X_train,y_train)
print(grid_pipe.best_score_, grid_pipe.best_params_)

Wall time: 1min 55s
0.8408337963248761 {'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2), 'lr__C': 1}


In [171]:
pred = grid_pipe.best_estimator_.predict(X_test)
acc = accuracy_score(y_test,pred)
print(f'CounterVectorizer + LogisticRegression 정확도 : {acc:.4f}')

CounterVectorizer + LogisticRegression 정확도 : 0.5071
